In [1]:
import math
import random
import string
import sys
import time
import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp, str_diff
from t2.utils import get_parser, join_sai, to_sai_str

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'false', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
params.my_device='cuda'

print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    
src.utils.CUDA = not params.cpu

env = build_env(params)

modules = build_transformer(env, params)
trainer = RealtimeTrainer(modules, env, params)


INFO - 01/09/23 21:30:59 - 0:00:00 - ============ Initialized logger ============
INFO - 01/09/23 21:30:59 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-6a98d561-c7bd-40b8-9668-ddb2a1195c67.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=False, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1, my_device='cuda')
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank 

INFO - 01/09/23 21:31:00 - 0:00:00 - Found 370 parameters in model.
INFO - 01/09/23 21:31:01 - 0:00:01 - Optimizers: model


In [2]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

print(trainer.single_act(A, A))

O<s>SO<s>X66IEJE66JXJE6XJ66J6E2626262E6EE6EXX6E66


In [3]:
start_time = time.time()

losses = []
iterations_number = 1000

for it in range(iterations_number):
        for _ in range(params.batch_size):
            if random.random() > 0.5:
                flag, loss = trainer.learn(A, B, X, 1)
            else:
                flag, loss = trainer.learn(X, A, B, 1)
        if flag:
            print(it, loss.item())

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


F:\workspace\ai\Transformers1\src\optim.py:72: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1418.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=O<s>SO<s>662J66666JXJ666J66<s>6E262666EE66EJ266J66E, score=0.06382978707551956
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=V<s>SORSSSSRXOSSSXSRSSRSR3RSRS2SSS2SUSURSSSSXSR, score=0.042553190141916275
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=V<s>SERSSSSSRISSSSA0SSRSSSSFSSSSSSRSRSSRRSSSSSR, score=0.06382978707551956
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

0 3.984987258911133


INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=<s>NSEMSSSXYS8SXA0A0SNNSSO4NR0ESSSRSSXSQSSSSS6R, score=0.08510638028383255
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=BOSERSSSISECSSA<s>A0SNSSRORES0S0SSSEOESRS0XSS6S, score=0.08510638028383255
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=BSSO<s>E66I<s>6EEEE<s>JOJXJ66O6E66IWE6JE6EB6<s>X6<s>E6I, score=0.042553190141916275
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=<s>RB6<s>X6EI6J<s>6EJ<s>JE66J6E<s>6660E6266E6E<s>6<s>6XXJ6W, score=0.042553190141916275
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=OOOO<s>666I0XH66<s>6TE6X666O66J6266XWE6E66EX6<s>J66, score=0.042553190141916275
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAA

1 3.5676419734954834


INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=O<s><s>O<s>XD6I0JB6BEXA<s>6XXEB<s>6EJBE3JXEE6EB<s>EXX0J6B, score=0.042553190141916275
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=BASO<s>SSSJ0JNSSA<s>A0ASS0S<s>AES0O<s>SX0SQ8SAASXAXSS, score=0.21276596188545227
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=E<s>SO<s>E0SS0ENS0A<s>A03XSAFUSASSEAXX0SX8SA0SXSXSS, score=0.1702127605676651
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=BASESASSS0AEOSEXA0<s>NX0U<s>AARSS0SXNARSSASSSAAXS, score=0.25531914830207825
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=<s><s>S0<s>ESQASE0SSAXA0ANAXB3AA00S<s>NS0AQXSASSQ0XSR, score=0.23404255509376526
INFO - 01/09/23 21:31:02 - 0:00:03 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

...

INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB, score=0.06382978707551956
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA, score=0.957446813583374
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA, score=0.957446813583374
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA, score=0.957446813583374
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA, score=0.957446813583374
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=BBBBBBBBBBBBBBBBBBBBBBBBB

92 0.9588229060173035


INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB, score=0.042553190141916275
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB, score=0.06382978707551956
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=ABBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB, score=0.042553190141916275
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB, score=0.042553190141916275
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA, score=0.957446813583374
INFO - 01/09/23 21:31:24 - 0:00:25 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAAAAAAAAAAAAAAA

93 0.8341002464294434


KeyboardInterrupt: 

In [ ]:
torch.cuda.get_device_name(next(trainers[0].modules['transformer'].te1.parameters()).device.index)

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(trainers[0].act(A))
print(trainers[0].act(B))

In [1]:
aa = [1,2,3]
bb = [5,6,7]

for a,b in zip(aa, bb):
    print(a, b)

1 5
2 6
3 7


In [15]:
s = set()

s.add(('a','b', ))

for a, b in s:
    print(a, b)

a b


In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()
